In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./iris.csv', sep=';')

In [4]:
# What is gonna be needed for decision tree algorithm
# Evaluate entropy for every feature and values for it
# After you pick best score, you branch out into 2 branches
# You stop at some n level
Y = df["species"]
Y.loc[Y >= 2]

100    2
101    2
102    2
103    2
104    2
105    2
106    2
107    2
108    2
109    2
110    2
111    2
112    2
113    2
114    2
115    2
116    2
117    2
118    2
119    2
120    2
121    2
122    2
123    2
124    2
125    2
126    2
127    2
128    2
129    2
130    2
131    2
132    2
133    2
134    2
135    2
136    2
137    2
138    2
139    2
140    2
141    2
142    2
143    2
144    2
145    2
146    2
147    2
148    2
149    2
Name: species, dtype: int64

In [5]:
# Rule format is (k, (condition))
class DecisionTree():
    def __init__(self, rootRule):
        self.rootRule = rootRule
        self.leftBranch = None
        self.rightBranch = None
class Branch():
    def __init__(self, rule):
        self.rule = rule
        self.leftBranch = None
        self.rightBranch = None

def evaluateFeature(df, label, feature):
    focusDf = df.loc[:,[feature, label]] # For now label name is hard coded, but we could have label feature parameter
    minVal = focusDf[feature].min()
    maxVal = focusDf[feature].max()
    bestGini = 1
    bestSplitVal = 0
    step = (maxVal - minVal)/15 # Evaluate 15 different values for split condition
    for i in range(1, 15): # We dont care about min and max split, in my opinion
        currentSplit = minVal + step * i
        leftSplit = focusDf.loc[(focusDf[feature] <= currentSplit)]
        rightSplit = focusDf.loc[(focusDf[feature] > currentSplit)]
        giniIndexLeft = 1
        giniIndexRight = 1
        #for left split
        for value in leftSplit[label].unique():
            #print(leftSplit.loc[(leftSplit["species"] == value)]["species"])
            classCount = leftSplit.loc[(leftSplit[label] == value)][label].count()
            totalCount = len(leftSplit)
            giniIndexLeft -= pow(classCount/totalCount,2) # gini calculation for left
        #for right split
        for value in rightSplit[label].unique():
            classCount = rightSplit.loc[(rightSplit[label] == value)][label].count()
            totalCount = len(rightSplit)
            giniIndexRight -= pow(classCount/totalCount,2) # gini calculation for right
        giniIndex = (giniIndexLeft + giniIndexRight)/2 # Get average of gini index
        #print(giniIndex)
        if giniIndex < bestGini:
            bestSplitVal = currentSplit
            bestGini = giniIndex
    #print(f"Gini: {bestGini} feature: {feature} split: {bestSplitVal}")
    return (feature, bestSplitVal, bestGini)

def branch(df, YLabel, k, n, verbose = True):
    if verbose:
        print(f"Branch on level {k} has df of size: {len(df)}")
    if k == n:
        # Reached bottom level, is not going to branch further
        label = df[YLabel].value_counts().index[0] # First index is gonna be the one with highest count in df
        thisBranch = Branch((k, ('END', label)))
        return thisBranch
    giniIndexMain = 1
    for value in df[YLabel].unique():
        classCount = df.loc[(df[YLabel] == value)][YLabel].count()
        totalCount = len(df)
        giniIndexMain -= pow(classCount/totalCount,2) # gini calculation for left
    if giniIndexMain == 0:
        # Branch is pure, no need for further branching
        label = df[YLabel].value_counts().index[0] # First index is gonna be the one with highest count in df
        thisBranch = Branch((k, ('END', label)))
        return thisBranch
    bestFeature = evaluateFeature(df, YLabel, df.columns[0])
    for col in df.columns[1:-1]: # Exclude first feature and last feature which is label
        result = evaluateFeature(df, YLabel, col)
        if result[2] < bestFeature[2]:
            bestFeature = result
    new_df1 = df.loc[(df[bestFeature[0]] <= bestFeature[1])]
    new_df2 = df.loc[(df[bestFeature[0]] > bestFeature[1])]
    thisBranch = Branch((k,bestFeature))
    thisBranch.leftBranch = branch(new_df1, YLabel, k + 1, n, verbose=verbose)
    thisBranch.rightBranch = branch(new_df2, YLabel, k + 1, n, verbose=verbose)
    return thisBranch

def DecisionTree_train(df, YLabel, n_levels = 3, verbose = True):
    # Get first feature and score
    k = 0
    bestFeature = evaluateFeature(df, YLabel, df.columns[0])
    for col in df.columns[1:-1]: # Exclude first feature and last feature which is label
        result = evaluateFeature(df, YLabel, col)
        if result[2] < bestFeature[2]:
            bestFeature = result
    #Split based on best feature into 2 new branches
    new_df1 = df.loc[(df[bestFeature[0]] <= bestFeature[1])]
    new_df2 = df.loc[(df[bestFeature[0]] > bestFeature[1])]
    tree = DecisionTree((k,bestFeature))
    #print(f"Gini: {bestFeature[2]} feature: {bestFeature[0]} split: {bestFeature[1]}")
    tree.leftBranch = branch(new_df1, YLabel, k + 1, n_levels, verbose=verbose)
    tree.rightBranch = branch(new_df2, YLabel, k + 1, n_levels, verbose=verbose)
    return tree
# Recursive function, takes df, iterates over all rows, returns list of predicted labels
def DecisionTree_predict(df, model : DecisionTree):
    firstRule = model.rootRule[1] # Get first condition 
    #print(firstRule)
    predictions = []
    for row in df.iterrows(): # Iterrows returns pairs of (index, Series)
        prediction = -1
        if row[1][firstRule[0]] <= firstRule[1]:
            prediction = branch_predict(row, model.leftBranch) # Go to left branch
        else:
            prediction = branch_predict(row, model.rightBranch)# Go to right branch
        predictions.append(prediction)
    return predictions
def branch_predict(row, model : Branch):
    rule = model.rule[1]
    if rule[0] == 'END':
        return rule[1]
    if row[1][rule[0]] <= rule[1]:
        prediction = branch_predict(row, model.leftBranch) # Go to left branch
    else:
        prediction = branch_predict(row, model.rightBranch) # Go to right branch
    return prediction

In [6]:
#Split df
train_X = df.sample(frac = 0.8)
test_X = df.drop(train_X.index)
test_X

,sepal_length,sepal_width,petal_length,petal_width,species
7,5.0,3.4,1.5,0.2,0
18,5.7,3.8,1.7,0.3,0
22,4.6,3.6,1.0,0.2,0
24,4.8,3.4,1.9,0.2,0
26,5.0,3.4,1.6,0.4,0
29,4.7,3.2,1.6,0.2,0
36,5.5,3.5,1.3,0.2,0
43,5.0,3.5,1.6,0.6,0
46,5.1,3.8,1.6,0.2,0
47,4.6,3.2,1.4,0.2,0


In [7]:
# Train the model
model = DecisionTree_train(train_X, "species")
model

Branch on level 1 has df of size: 40
Branch on level 1 has df of size: 80
Branch on level 2 has df of size: 43
Branch on level 3 has df of size: 1
Branch on level 3 has df of size: 42
Branch on level 2 has df of size: 37
Branch on level 3 has df of size: 36
Branch on level 3 has df of size: 1


In [12]:
# Predict
predictions = DecisionTree_predict(test_X, model)
set(predictions)

{0, 1, 2}

In [13]:
# Evaluate
import sklearn.metrics
sklearn.metrics.accuracy_score(test_X.species, predictions)

0.9

In [14]:
evaluateFeature(df, "species", 'sepal_length')

('sepal_length', 4.78, 0.3312457947311216)

In [15]:
evaluateFeature(df, "species", 'sepal_width')

('sepal_width', 3.9200000000000004, 0.3330831300431601)

In [16]:
evaluateFeature(df, "species", 'petal_length')

('petal_length', 2.18, 0.25)

In [17]:
evaluateFeature(df, "species", 'petal_width')

('petal_width', 0.74, 0.25)

In [18]:
df = pd.read_csv("./titanic_preprocessed.csv", sep=",", index_col='PassengerId')

In [19]:
df = df[[c for c in df if c not in ['Survived']] + ['Survived']]

In [20]:
df.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Survived'],
      dtype='object')

In [21]:
train_X = df.sample(frac = 0.8)
test_X = df.drop(train_X.index)
test_X

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Survived
PassengerId,,,,,,,,,,,
6,3,0.0,29.699118,0,0,8.4583,7.0,0,1,0,0
11,3,1.0,4.000000,1,1,16.7000,6.0,0,0,1,1
17,3,0.0,2.000000,4,1,29.1250,7.0,0,1,0,0
18,2,0.0,29.699118,0,0,13.0000,7.0,0,0,1,1
30,3,0.0,29.699118,0,0,7.8958,7.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
880,1,1.0,56.000000,0,1,83.1583,2.0,1,0,0,1
881,2,1.0,25.000000,0,1,26.0000,7.0,0,0,1,1
887,2,0.0,27.000000,0,0,13.0000,7.0,0,0,1,0


In [22]:
# Train the model
model = DecisionTree_train(train_X, "Survived")
model

Branch on level 1 has df of size: 709
Branch on level 2 has df of size: 708
Branch on level 3 has df of size: 707
Branch on level 3 has df of size: 1
Branch on level 2 has df of size: 1
Branch on level 1 has df of size: 2


In [23]:
predictions = DecisionTree_predict(test_X, model)
set(predictions)

{0, 1}

In [24]:
sklearn.metrics.accuracy_score(test_X.Survived, predictions)

0.651685393258427

In [26]:
for i in range(1, 16):
    model = DecisionTree_train(train_X, "Survived", n_levels=i, verbose=False)
    predictions = DecisionTree_predict(test_X, model)
    print(f"n_levels = {i} accuracy = {sklearn.metrics.accuracy_score(test_X.Survived, predictions)}")

n_levels = 1 accuracy = 0.651685393258427
n_levels = 2 accuracy = 0.651685393258427
n_levels = 3 accuracy = 0.651685393258427
n_levels = 4 accuracy = 0.651685393258427
n_levels = 5 accuracy = 0.651685393258427
n_levels = 6 accuracy = 0.651685393258427
n_levels = 7 accuracy = 0.8202247191011236
n_levels = 8 accuracy = 0.8202247191011236
n_levels = 9 accuracy = 0.8202247191011236
n_levels = 10 accuracy = 0.8202247191011236
n_levels = 11 accuracy = 0.8202247191011236
n_levels = 12 accuracy = 0.8202247191011236
n_levels = 13 accuracy = 0.8202247191011236
n_levels = 14 accuracy = 0.8426966292134831
n_levels = 15 accuracy = 0.8426966292134831


In [ ]:
model = DecisionTree_train(train_X, "Survived", n_levels=50, verbose=False) # No limit on n levels
predictions = DecisionTree_predict(test_X, model)
print(f"n_levels = Not limited, accuracy = {sklearn.metrics.accuracy_score(test_X.Survived, predictions)}")

KeyboardInterrupt: 